### Active using of sklearn (data - https://archive.ics.uci.edu/ml/datasets/student+performance)

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

from sklearn.linear_model import Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.grid_search import GridSearchCV
from sklearn.learning_curve import validation_curve

from sklearn.metrics import mean_squared_error

%matplotlib inline

In [12]:
df = pd.read_csv('student-por.csv', ';')
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


**1. Разбейте выборку случайным образом на обучение и контроль (train_test_split из sklearn). В обучение попадает 30% выборки, в контроль 70%. Разбейте контроль в отношении 4:3. Большая часть - контрольная выборка, которой вы будете пользоваться. Меньшая часть - скрытый контроль - используется только в последнем пункте.**

In [13]:
# X - набор номеров, присвоенных студентам, длина которого равна количеству строк в исходной таблице
X = np.arange(df.shape[0])

# разбиваем выборку на обучение и контроль в заданном соотношении
X_train, X_test = train_test_split(X, test_size=0.7)

# разбиваем контроль (X_test) в отношении 4:3, т.е. в используемую контрольную выборку попадет ~57%
X_control, X_concealed = train_test_split(X_test, train_size=0.57)

print(X_train, len(X_train))
print(X_control, len(X_control))
print(X_concealed, len(X_concealed))

[198 402 106 647 628 286 122  10 165  98 154 378 268 456 585 179 316 133
  28 118 487 211 363  93 618 382 377 362 207 258 152 477 423 540 393 276
  24 643 575 615 501 199 101 480 157 347 564 542 411 361 226 103 613 344
 240 313 427 129 174 360 215 354 593 150 473 195  64 572 219 431 381 320
 170 637 371 574 310 331  15 588 530  60 337  73  88  66   1 434 153  51
 285   9 221 506 399 141 638  40 418 100 417 356  31 486 343 281 470  54
  61 351 619 376 490 648 482 188 242 151 412 584 192 159 102 340 367 394
  22 213  71 305 449 504 358 339 142 557  70 544 369 586 645 334 121 111
 620 161 261 420  74 350  53 311 346  69 295 600 566 636 272 548  65  46
 169 255  56 306 236 232 357 446 368 205 125 395 302  32 612 563 373 625
 561 132 303 626  82 300 227 148 639 238 488 336 546 180] 194
[595 244 537 633 349 496 342 407 443 251 265 499 383 599 370 405 217 182
 601 568 515 536 527 388  48 436 508 214 279 321 115 428 193 296 522 327
 181  76 535  58 492 366 243 228 220 173 607 410 525 287 509 3

**2. Задача сводится к предсказанию целевого признака G3 (не используя признаки G1 и G2) по обучающей выборке. С учетом предыдущей части проекта определитесь с пространством признаков, с которым вы будете работать. Можно генерировать новые признаки из уже существующих.**

В первой части проекта мной были проанализированы некоторые признаки, в ходе чего было выявлено, что на итоговую оценку (G3) в среднем влияют выбор школы (school), количество пропусков (absences), желание получить высшее образование (higher), наличие у родителей высшего образования (Medu, Fedu), количество учебных часов в неделю (studytime), наличие домашнего интернета (internet), наличие дополнительных и платных дополнительных занятий (schoolsup, paid), употребление алкоголя в будние дни (Dalc), наличие романтических отношений (romantic). С ними я и буду работать.

In [14]:
df = df[['school', 'Medu', 'Fedu', 'studytime', 'schoolsup', 'paid', 'higher', 
         'internet', 'romantic', 'Dalc', 'absences', 'G3']]
df.head()

,school,Medu,Fedu,studytime,schoolsup,paid,higher,internet,romantic,Dalc,absences,G3
0,GP,4,4,2,yes,no,yes,no,no,1,4,11
1,GP,1,1,2,no,no,yes,yes,no,1,2,11
2,GP,1,1,2,yes,no,yes,yes,no,2,6,12
3,GP,4,2,3,no,no,yes,yes,yes,1,0,14
4,GP,3,3,2,no,no,yes,no,no,1,0,13


In [15]:
# для текстовых признаков применим one-hot-кодирование, чтобы все значения в таблице были численными
df = pd.get_dummies(df)
df.head()

,Medu,Fedu,studytime,Dalc,absences,G3,school_GP,school_MS,schoolsup_no,schoolsup_yes,paid_no,paid_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,4,4,2,1,4,11,1,0,0,1,1,0,0,1,1,0,1,0
1,1,1,2,1,2,11,1,0,1,0,1,0,0,1,0,1,1,0
2,1,1,2,2,6,12,1,0,0,1,1,0,0,1,0,1,1,0
3,4,2,3,1,0,14,1,0,1,0,1,0,0,1,0,1,0,1
4,3,3,2,1,0,13,1,0,1,0,1,0,0,1,1,0,1,0


In [16]:
# формируем таблицы, соответствующие проведенным выборкам студентов
df_original = df.ix[X_train]
df_control = df.ix[X_control]
df_concealed = df.ix[X_concealed]

# здесь формируем для каждой выборки признаковое описание и результат (значение G3)
df_train = df_original.drop('G3', axis=1)
df_target = df_original.G3
df_control_features = df_control.drop('G3', axis=1)
df_control_target = df_control.G3
df_concealed_features = df_concealed.drop('G3', axis=1)
df_concealed_target = df_concealed.G3

**3. Обратите внимание, что задачу можно рассматривать как задачу классификации, а можно - как задачу регрессии. Настроить как минимум 5 различных классификаторов, подобрать гиперпараметры для каждого с помощью кросс-валидации. Построить графики зависимости качества на кросс-валидации от значений гиперпараметров.**

Я еще в первой части проекта рассматривал задачу как задачу регрессии. Поэтому здесь я буду отталкиваться от этого и настраивать регрессоры.

In [17]:
# массив регрессоров, гиперпараметры к которым будут настраиваться
regs = [Ridge(), Lasso(), RandomForestRegressor(), DecisionTreeRegressor(), KNeighborsRegressor()]

# массив настраиваемых параметров и их возможных значений
regs_parameters = [{'alpha': np.arange(1, 30, 0.5)}, {'alpha': np.linspace(0, 0.1, 25)[1:]},
                   {'n_estimators': np.arange(10, 61, 2)}, {'min_samples_leaf': np.arange(3, 61, 3)}, 
                   {'n_neighbors': np.arange(2, 41, 2)}]

In [21]:
# с помощью функции ниже можно будет построить графики зависимости качества на кросс-валидации для всех
# регрессоров сразу

def plot_validation_curve(reg, reg_parameter, title):
    param_name = list(reg_parameter.keys())[0]
    param_range = list(reg_parameter.values())[0]
    train_scores, test_scores = validation_curve(
        reg, df_train, df_target, param_name=param_name, param_range=param_range,
        cv=5, scoring='mean_squared_error')
    
    # среднее значение берется с минусом, так как оно получается отрицательным. В гугле нашел, что
    # это норма; возможно (я могу быть не прав), это происходит из-за того, что в прописанных
    # в sklearn функциях ошибка максимизируется при анализе качества, и поэтому разработчики
    # решили не напрягаться и стали рассматривать MSE с минусом. В общем, надо смотреть на модуль
    train_scores_mean = np.sqrt(-np.mean(train_scores, axis=1))
    train_scores_std = np.sqrt(np.std(train_scores, axis=1))
    test_scores_mean = np.sqrt(-np.mean(test_scores, axis=1))
    test_scores_std = np.sqrt(np.std(test_scores, axis=1))
    
    plt.figure()
    plt.title(title)
    plt.xlabel(param_name)
    plt.ylabel('Score')
    plt.plot(param_range, train_scores_mean, label='Training score', color='r')
    plt.fill_between(param_range, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.2, color='r')
    plt.plot(param_range, test_scores_mean, label='Cross-validation score',
                 color='g')
    plt.fill_between(param_range, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.2, color='g')
    plt.legend(loc='best')
    return plt

# теперь найдем нужные гиперпараметры, попутно нарисовав график
for i in range(len(regs)):
    reg = regs[i]
    reg_parameter = regs_parameters[i]
    grid_search = GridSearchCV(reg, reg_parameter, cv=5)
    grid_search.fit(df_train, df_target)
    print(grid_search.best_params_)
    
    
    # в исходном массиве с регрессорами заменим каждый элемент на соответствующий регрессор с найденным
    # значением гиперпараметра
    regs[i] = grid_search.best_estimator_

{'alpha': 16.5}
{'alpha': 0.029166666666666667}
{'n_estimators': 22}
{'min_samples_leaf': 18}
{'n_neighbors': 18}


**4. Оценить качество классификаторов на контроле (например, в качестве функционала ошибок можно использовать RMSE). Выбрать наилучший.**

In [45]:
# новый массив с предсказаниями регрессоров
reg_preds = []

# предсказываем и считаем RMSE
for reg in regs:
    reg_pred = reg.predict(df_control_features)
    reg_preds.append(reg_pred)
    print(np.sqrt(mean_squared_error(df_control_target, reg_pred)))

3.0618385592
3.04858313249
3.20571073172
3.24832343422
3.24325366728


Видно, что на данном разбиении из выбранных регрессоров лучшее качество имеют Ridge и Lasso (первый чуть уступает второму), остальные регрессоры работают чуть хуже (причем решающее дерево и kNN работают почти одинаково).

**5. Рассмотреть простые композиции алгоритмов (хотя бы 2), проверить их качество.**

In [48]:
# сначала берем результаты лучших регрессоров с одинаковыми весами
new_pred1 = 0.5 * reg_preds[0] + 0.5 * reg_preds[1]

# здесь отдаем приоритет самому лучшему регрессору (Lasso), а те, что похуже, включаем с очень
# маленькими весами
new_pred2 = 0.7 * reg_preds[1] + 0.1 * (reg_preds[2] + reg_preds[3] + reg_preds[4])

print(np.sqrt(mean_squared_error(df_control_target, new_pred1)))
print(np.sqrt(mean_squared_error(df_control_target, new_pred2)))

3.05056539043
3.03902318756


Оказалось, что последняя композиция, состоящая из Lasso с большим весом и из Random Forest, решающего дерева и kNN с очень маленькими весами (далее - композиция 2) работает на данной выборке лучше всех рассмотренных алгоритмов. Стоит также заметить, что относительно хорошее качество показывает и композиция из Lasso и Ridge с одинаковыми весами (далее - композиция 1).

**6. Выбрать финальный классификатор, посчитать качество на скрытом контроле.**

In [52]:
# считаем качество композиции 2 на скрытом контроле

final_pred = 0.7 * regs[1].predict(df_concealed_features) + \
     0.1 * (regs[2].predict(df_concealed_features) + regs[3].predict(df_concealed_features) + \
           regs[4].predict(df_concealed_features))
    
print(np.sqrt(mean_squared_error(df_concealed_target, final_pred)))

2.67204253144


**7. Сделать выводы.**

Итак, я рассмотрел работу регрессоров Ridge, Lasso, Random Forest, Decision Tree и kNN на данной случайной выборке, а также двух композиций этих алгоритмов с разными весами. Было установлено, что Ridge и Lasso работают лучше остальных алгоритмов (с небольшим преимуществом последнего), Decision Tree и kNN хуже Random Forest и почти одинаково. Композиции с упором на лучшие алгоритмы дают хорошие результаты, причем композиция 2 даже обгоняет по качеству всё остальное.